In [1]:
#pip install gazpacho

In [2]:
from gazpacho import get, Soup
import numpy
import pandas
import requests, bs4
import re, os
import time

In [3]:
# Verifying that the page request is valid
from gazpacho import get, Soup

url = 'https://www.imperialyeast.com/yeast-strains/house'

# fetch step
html = get(url)

# extract step
soup = Soup(html)

In [4]:
# use find() to parse a specific HTML
gaz_data = soup.find('h1')
gaz_data

[<h1 class="pre-animated hero-step text-h1 -md:text-h1-mobile">House</h1>,
 <h1 class="relative h0-sm space-x-10 -lg:space-x-3.5 -lg:max-w-sm max-w-3xl flex flex-wrap mt-3.5 sm:mt-5 mx-auto justify-center">
   <span>See</span>
   <span>What's</span>
   <span>Brewing</span>
 </h1>]

### Pull The Strain Name

In [5]:
# Pull The Strain Name

def pull_strain_name(soup):
    strain_name = soup.find('h1')
    x = str(strain_name)
    start = x.find('>')
    end = x.find('</h1>')
    strain = x[start+1:end]
    # print(strain)
    return strain

In [6]:
pull_strain_name(soup)

'House'

### Pull The Strain Number

In [7]:
# Pull The Strain Number

def pull_strain_number(soup):
    strain_number = soup.find('h2')
    x = str(strain_number)
    start = x.find('<h2>')
    number = x[start+4:start+7]
    # print(number)
    return number

In [8]:
pull_strain_number(soup)

'A01'

### Pull The Description

In [9]:
# Pull and save the description

def pull_strain_description(soup):
    description = soup.find('p')
    #print(description)
    x= str(description)
    start = x.find('mt-2.5">')
    end = x.find('<p class="mt-2.5">')
    description = x[start+8:end-6]
    # print(description)
    return description

In [10]:
pull_strain_description(soup)

'The best of both worlds, House is clean and allows malt and hops to shine. This strain is extremely versatile and flocculent enough to drop out of the beer quickly. Best strain choice for any and all English- inspired recipes, however, does extremely well in hopped up American styles too. House is clean at cold temperatures with increased esters as fermentation temperatures rise.'

### Get Stats on Strain

In [11]:
def pull_stats(soup):
    raw = soup.find('h6')
    x= str(raw)
    print(x)
    fs = x.find('<h6 class="uppercase mt-1">', x.find('<h6 class="uppercase mt-1">')+1)
    flocculation = x[fs+27:fs+40]
    # print(flocculation)

    attens = x.find('%')
    attenuation = x[attens-8:attens+1]
    # print(attenuation)

    abve = x.find('%') + 75
    abv = x[abve:abve+5]
    # print(abv)

    temps = x.find('°')
    tempe = x.find('°')
    temp = x[temps-6:tempe+12]
    # print(temp)

    return(flocculation, attenuation, abv, temp)

In [12]:
pull_stats(soup)

[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="text-white uppercase mb-11 xl:translate-x-3.5 font-semibold">More Products</h6>, <h6 class="uppercase mt-1">62-70 °F / 17-21 °C</h6>, <h6 class="uppercase mt-1">medium</h6>, <h6 class="uppercase mt-1">high</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">73-75%</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">10%</h6>, <h6 class="mb-1 text-black">A07</h6>]


('medium</h6>, ', '1">73-75%', '">10%', '62-70 °F / 17-21 °')

### Pull all the urls

In [13]:
import pandas as pd
url = pd.read_csv('data/imperial_urllist.csv')

In [14]:
url_list = url['Address'].to_list()

In [15]:
url_list

['https://www.imperialyeast.com/yeast-strains/barbarian',
 'https://www.imperialyeast.com/yeast-strains/bartleby',
 'https://www.imperialyeast.com/yeast-strains/brett-brux',
 'https://www.imperialyeast.com/yeast-strains/brett-clausenii',
 'https://www.imperialyeast.com/yeast-strains/brett-drei',
 'https://www.imperialyeast.com/yeast-strains/brett-lambicus',
 'https://www.imperialyeast.com/yeast-strains/bubbles',
 'https://www.imperialyeast.com/yeast-strains/cablecar',
 'https://www.imperialyeast.com/yeast-strains/capri',
 'https://www.imperialyeast.com/yeast-strains/champion',
 'https://www.imperialyeast.com/yeast-strains/citrus',
 'https://www.imperialyeast.com/yeast-strains/corporate',
 'https://www.imperialyeast.com/yeast-strains/darkness',
 'https://www.imperialyeast.com/yeast-strains/dieter',
 'https://www.imperialyeast.com/yeast-strains/dry-hop',
 'https://www.imperialyeast.com/yeast-strains/fillibuster',
 'https://www.imperialyeast.com/yeast-strains/flagship',
 'https://www.impe

In [16]:
import pandas as pd

def loadTable(soup):
    strain = pull_strain_name(soup)
    #pullMetrics(soup)
    number = pull_strain_number(soup)
    flocculation, attenuation, abv, temp = pull_stats(soup)
    description = pull_strain_description(soup)
    entry = {'Strain': [strain],
             'Number' : [number],
        'Attenuation' : [attenuation],
        'Flocculation' : [flocculation],
        'Alcohol_Tolerance' : [abv],
        'Fermentation_Temperature' : [temp],
        'Description' : [description]}
    df = pd.DataFrame(entry)
    return df

In [17]:
loadTable(soup)

[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="text-white uppercase mb-11 xl:translate-x-3.5 font-semibold">More Products</h6>, <h6 class="uppercase mt-1">62-70 °F / 17-21 °C</h6>, <h6 class="uppercase mt-1">medium</h6>, <h6 class="uppercase mt-1">high</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">73-75%</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">10%</h6>, <h6 class="mb-1 text-black">A07</h6>]


,Strain,Number,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,Description
0,House,A01,"1"">73-75%","medium</h6>,",""">10%",62-70 °F / 17-21 °,"The best of both worlds, House is clean and al..."


In [18]:
df = loadTable(soup)
df1 = df
df1

[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="text-white uppercase mb-11 xl:translate-x-3.5 font-semibold">More Products</h6>, <h6 class="uppercase mt-1">62-70 °F / 17-21 °C</h6>, <h6 class="uppercase mt-1">medium</h6>, <h6 class="uppercase mt-1">high</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">73-75%</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">10%</h6>, <h6 class="mb-1 text-black">A07</h6>]


,Strain,Number,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,Description
0,House,A01,"1"">73-75%","medium</h6>,",""">10%",62-70 °F / 17-21 °,"The best of both worlds, House is clean and al..."


In [19]:
for i in url_list:
  print(i)
  # fetch step
  html = get(i)

  # extract step
  soup = Soup(html)
  df = loadTable(soup)
  df1 = pd.concat([df1, df])

https://www.imperialyeast.com/yeast-strains/barbarian
[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="text-white uppercase mb-11 xl:translate-x-3.5 font-semibold">More Products</h6>, <h6 class="uppercase mt-1">62-70 °F / 17-21 °C</h6>, <h6 class="uppercase mt-1">medium</h6>, <h6 class="uppercase mt-1">medium</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">73-74%</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">10%</h6>, <h6 class="mb-1 text-black">A24</h6>, <h6 class="mb-1 text-black">A38</h6>]
https://www.imperialyeast.com/yeast-strains/bartleby
[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class

[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="text-white uppercase mb-11 xl:translate-x-3.5 font-semibold">More Products</h6>, <h6 class="uppercase mt-1">60-73 °F / 16-23 °C</h6>, <h6 class="uppercase mt-1">medium</h6>, <h6 class="uppercase mt-1">medium</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">73-77%</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">10%</h6>, <h6 class="mb-1 text-black">A07</h6>]
https://www.imperialyeast.com/yeast-strains/darkness
[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="text-white uppercase mb-11 xl:

[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="text-white uppercase mb-11 xl:translate-x-3.5 font-semibold">More Products</h6>, <h6 class="uppercase mt-1">62-70 °F / 17-21 °C</h6>, <h6 class="uppercase mt-1">medium</h6>, <h6 class="uppercase mt-1">high</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">73-75%</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">10%</h6>, <h6 class="mb-1 text-black">A07</h6>]
https://www.imperialyeast.com/yeast-strains/hygge
[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="text-white uppercase mb-11 xl:trans

[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="text-white uppercase mb-11 xl:translate-x-3.5 font-semibold">More Products</h6>, <h6 class="uppercase mt-1">65-100 °F / 18-38 °C</h6>, <h6 class="uppercase mt-1">medium</h6>, <h6 class="uppercase mt-1">medium-high</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">75-85%</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">10%</h6>, <h6 class="mb-1 text-black">A24</h6>, <h6 class="mb-1 text-black">A38</h6>]
https://www.imperialyeast.com/yeast-strains/mangosteeni
[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For<

[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="text-white uppercase mb-11 xl:translate-x-3.5 font-semibold">More Products</h6>, <h6 class="uppercase mt-1">64-74 °F / 18-23 °C</h6>, <h6 class="uppercase mt-1">medium</h6>, <h6 class="uppercase mt-1">medium</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">76-78%</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">12%</h6>, <h6 class="mb-1 text-black">B48</h6>, <h6 class="mb-1 text-black">B56</h6>]
https://www.imperialyeast.com/yeast-strains/pub
[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 clas

[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="text-white uppercase mb-11 xl:translate-x-3.5 font-semibold">More Products</h6>, <h6 class="uppercase mt-1">62-72 °F / 17-22 °C</h6>, <h6 class="uppercase mt-1">low</h6>, <h6 class="uppercase mt-1">low-medium</h6>, <h6 class="uppercase mt-1">positive</h6>, <h6 class="uppercase mt-1">72-76%</h6>, <h6 class="uppercase mt-1">negative</h6>, <h6 class="uppercase mt-1">10%</h6>, <h6 class="mb-1 text-black">L28</h6>, <h6 class="mb-1 text-black">A34</h6>, <h6 class="mb-1 text-black">W12</h6>, <h6 class="mb-1 text-black">A43</h6>]
https://www.imperialyeast.com/yeast-strains/workhorse
[<h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Strains For</h6>, <h6 class="uppercase font-semibold">Yeast Str

In [20]:
df1

,Strain,Number,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,Description
0,House,A01,"1"">73-75%","medium</h6>,",""">10%",62-70 °F / 17-21 °,"The best of both worlds, House is clean and al..."
0,Barbarian,A04,"1"">73-74%","medium</h6>,",""">10%",62-70 °F / 17-21 °,Commonly referred to as the conan yeast strain...
0,Bartleby,A46,"1"">75-85%","low</h6>, <h6",""">10%",68-98 °F / 20-37 °,Bartleby is a hornindal Kveik strain that is a...
0,Brett Brux,W13,"1"">55-85%","medium</h6>,",""">12%",64-74 °F / 18-23 °,Brux is a classic Brettanomyces bruxellensis s...
0,Brett Clausenii,W12,"1"">50-60%","medium</h6>,",""">12%",70-85 °F / 21-29 °,Claussenii is a Brettanomyces strain favored f...
0,Brett Drei,W05,"1"">56-85%",low-high</h6>,""">10%",70-85 °F / 21-29 °,A strain of Brettanomyces bruxellensis known t...
0,Brett Lambicus,W10,"1"">55-85%","medium</h6>,",""">12%",70-85 °F / 21-29 °,Lambicus is a Brettanomyces bruxellensis strai...
0,Bubbles,A40,""">85-100%","low</h6>, <h6",""">10%",57-77 °F / 14-25 °,"The clean profile of the yeast, especially whe..."
0,Cablecar,L05,"1"">71-75%","medium</h6>,",""">10%",55-65 °F / 13-18 °,Cablecar can produce clean pseudo lagers at al...
0,Capri,I22,"1"">68-74%","low</h6>, <h6",""">10%",68-86 °F / 20-30 °,Capri is a small-celled warrior fit to bring o...


# Pulling all the urls that are needed

In [21]:
df1.to_csv('Imperial_Strains.csv')